In [218]:
#1.Pick a random news article (preferably with many entity mentions) from your Webhose dataset 
import json

#read data from json file
json_data=open("IBM_english.json").readlines()
#print(json_data)

#Output: print the number of articles, titles and publish dates of the first 100 articles.
feeds_read_from_file = []
for line in json_data:
    feeds_read_from_file.append(json.loads(line))
#print(len(feeds_read_from_file))

#Pick a random news article from your Webhose dataset 
article = feeds_read_from_file[9]['text']
article_url = feeds_read_from_file[9]['url']

print(article)
print(article_url)

#len(article['text'])

#Pick a random test article from your Webhose dataset 
test = feeds_read_from_file[3]['text']
#len(article['text'])
print(test)


Linda Fister's story with IBM beyond mirrors the onset and boom of computers. Exeter Township, PA —
The big metal box with blinking lights transfixed 12-year-old Linda Fister.
It was the first computer she had seen, and Fister stood staring in wonder at the state-of-the-art display at the Museum of Science and Industry in Chicago.
“It piqued my curiosity,” she said. “I thought, 'This is the future.' ”
There and then, she decided it would be her future.
A retired computer programmer, Fister, now 75, of Exeter Township was the only woman in her class when she started programming close to 60 years ago. In recognition of Day of the Programmer, an international professional day celebrated Sept. 13, she recently reflected on what it was like to be a woman in the early days of programming.
A few years after that decisive museum trip, Fister, who grew up in a suburb of the Windy City, walked into the Chicago office of International Business Machines, or IBM, to ask for a job. A precocious teen

In [173]:
#Tokenize sentences 

from nltk import word_tokenize, sent_tokenize, ngrams, pos_tag, RegexpParser
from collections import Counter

#from nltk import tokenize
# tokenize sentences
sentences = sent_tokenize(article)
print('# of sentences:',len(sentences))

count=0
for line in sentences:
    print("sentence:",count)
    print(line)
    count=count+1


# of sentences: 56
sentence: 0
Linda Fister's story with IBM beyond mirrors the onset and boom of computers.
sentence: 1
Exeter Township, PA —
The big metal box with blinking lights transfixed 12-year-old Linda Fister.
sentence: 2
It was the first computer she had seen, and Fister stood staring in wonder at the state-of-the-art display at the Museum of Science and Industry in Chicago.
sentence: 3
“It piqued my curiosity,” she said.
sentence: 4
“I thought, 'This is the future.'
sentence: 5
”
There and then, she decided it would be her future.
sentence: 6
A retired computer programmer, Fister, now 75, of Exeter Township was the only woman in her class when she started programming close to 60 years ago.
sentence: 7
In recognition of Day of the Programmer, an international professional day celebrated Sept. 13, she recently reflected on what it was like to be a woman in the early days of programming.
sentence: 8
A few years after that decisive museum trip, Fister, who grew up in a suburb of

In [174]:
print(sentences[0][26:29])
print(sentences[8][130:160],sentences[8][166:169])
print(sentences[11][38:41])
print(sentences[23][62:65])
print(sentences[27][174:177])
print(sentences[31][67:70])
print(sentences[44][60:90])
print(sentences[51][18:30])


IBM
International Business Machine IBM
IBM
IBM
IBM
IBM
Allentown-area business school
organization


In [195]:
#update TRAIN_DATA with annotations of entities (PERSON, LOCATION, or ORGANIZATION) from each sentence in the selected news article
TRAIN_DATA = [
    (sentences[0], {"entities": [(26, 29, "ORG")]}),
    (sentences[8], {"entities": [(130, 160, "ORG"), (166, 169, "ORG")]}),
    (sentences[11], {"entities": [(38, 41, "ORG")]}),
    (sentences[23], {"entities": [(62, 65, "ORG")]}),
    (sentences[27], {"entities": [(174, 177, "ORG")]}),
    (sentences[31], {"entities": [(67, 70, "ORG")]}),
    (sentences[44], {"entities": [(60, 90, "ORG")]}),
    (sentences[51], {"entities": [(18, 30, "ORG")]}),
   
]

In [175]:
#!pip install spacy 

!pip install spacy 

In [176]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [177]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


In [178]:
#run spaCy_NER function to generate trained_nlp model
nlp=spacy.load('en_core_web_sm')

In [179]:
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")


In [180]:
# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])
        

In [181]:
#Train NER
n_iter=100
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):  # only train NER
    # reset and initialize the weights randomly – but only if we're
    # training a new model
    #if model is None:
    nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                losses=losses,
            )
        print("Losses", losses)


Losses {'ner': 36.32638852158561}
Losses {'ner': 19.351716697216034}
Losses {'ner': 37.23130480153486}
Losses {'ner': 14.838703060289845}
Losses {'ner': 19.930521067144582}
Losses {'ner': 17.210292600793764}
Losses {'ner': 12.752193732536398}
Losses {'ner': 13.312805344403387}
Losses {'ner': 7.127047104476333}
Losses {'ner': 15.518727780610789}
Losses {'ner': 6.645637699930376}
Losses {'ner': 11.328733414165981}
Losses {'ner': 12.729191844220964}
Losses {'ner': 11.747025308748562}
Losses {'ner': 8.21453882926653}
Losses {'ner': 5.004280650723331}
Losses {'ner': 9.140535384089162}
Losses {'ner': 5.666354907825735}
Losses {'ner': 8.4677728091483}
Losses {'ner': 14.66655692178756}
Losses {'ner': 6.927038345364053}
Losses {'ner': 8.627055278320768}
Losses {'ner': 6.749617352536006}
Losses {'ner': 7.688319371201942}
Losses {'ner': 3.0982062307591}
Losses {'ner': 5.901205892844587}
Losses {'ner': 5.75887910999927}
Losses {'ner': 3.441872582085125}
Losses {'ner': 3.910064725679831}
Losses {'n

In [182]:
# test the trained model 
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('IBM', 'ORG')]
Tokens [('No', '', 2), ('flirtation', '', 2), ('.', '', 2), ('”', '', 2), ('\n', '', 2), ('Some', '', 2), ('of', '', 2), ('her', '', 2), ('male', '', 2), ('counterparts', '', 2), ('were', '', 2), ('skeptical', '', 2), (',', '', 2), ('but', '', 2), ('she', '', 2), ('earned', '', 2), ('their', '', 2), ('respect', '', 2), ('by', '', 2), ('keeping', '', 2), ('up', '', 2), ('to', '', 2), ('pace', '', 2), ('in', '', 2), ('Formula', '', 2), ('Translation', '', 2), (',', '', 2), ('better', '', 2), ('known', '', 2), ('as', '', 2), ('Fortran', '', 2), (',', '', 2), ('and', '', 2), ('RPG', '', 2), (',', '', 2), ('an', '', 2), ('IBM', 'ORG', 3), ('proprietary', '', 2), ('language', '', 2), ('for', '', 2), ('business', '', 2), ('applications', '', 2), ('.', '', 2)]
Entities [('IBM', 'ORG')]
Tokens [('She', '', 2), ('did', '', 2), ('and', '', 2), ('landed', '', 2), ('an', '', 2), ('internship', '', 2), ('with', '', 2), ('IBM', 'ORG', 3), ('.', '', 2)]
Entities [('Internatio

In [183]:
def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA, 20)


Statring iteration 0
{'ner': 107.25856900215149}
Statring iteration 1
{'ner': 18.939542730544872}
Statring iteration 2
{'ner': 12.61003939968441}
Statring iteration 3
{'ner': 7.810625788726654}
Statring iteration 4
{'ner': 3.2564568093882533}
Statring iteration 5
{'ner': 1.3145751149162712}
Statring iteration 6
{'ner': 1.6307507148000429}
Statring iteration 7
{'ner': 9.620695947077055}
Statring iteration 8
{'ner': 4.605250749938947}
Statring iteration 9
{'ner': 4.501363718152714}
Statring iteration 10
{'ner': 2.7076242212399593}
Statring iteration 11
{'ner': 2.1860470380960244}
Statring iteration 12
{'ner': 4.678292948178114}
Statring iteration 13
{'ner': 2.232514249616217}
Statring iteration 14
{'ner': 1.967812652715275}
Statring iteration 15
{'ner': 2.0556907426367887}
Statring iteration 16
{'ner': 3.429128123671626}
Statring iteration 17
{'ner': 2.1033418445967227}
Statring iteration 18
{'ner': 0.4892249814454363}
Statring iteration 19
{'ner': 0.15688536596215294}


In [184]:

# Save our trained Model
modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)

#use trained_nlp to test entity recognition on another random news article from Webhose and print results to output

test_text = input("Enter your testing text: ")
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Enter your Model Name: IBM
Enter your testing text: Someday, a new class of semiconductor companies will assemble their products Lego-style, mixing and matching dice from multiple companies. It’s a huge change that’s still a long way off. More than 146 people from 80 companies signed up for an event last week at IBM’s Almaden Research Center to take some small steps in this direction. It was the third major face-to-face this year for the Open Domain-Specific Architecture (ODSA) group, following others hosted by Samsung and Intel. In a sign of growing interest, a similar event a year ago drew fewer than half as many people from fewer than half as many companies. “We don’t know if we have the right solution, but we are reasonably confident we have the right problem,” said Bapi Vinnakota, a Netronome engineer who leads ODSA. The problem is that chiplets lack open interoperable interfaces that would enable anyone to assemble them into products. In an effort to create such interfaces, ODSA 

In [186]:
#3. Choose  one  of  the  Information Extraction  services  below,  setup  a free  account, 

pip install --upgrade "ibm-watson>=3.4.0"

     |████████████████████████████████| 307kB 3.0MB/s eta 0:00:01
     |████████████████████████████████| 204kB 19.9MB/s eta 0:00:01
  Stored in directory: /Users/rissacao/Library/Caches/pip/wheels/40/73/8a/809e738908cd1b0d34a383219c49ede9f9fba487584ad3377f
  Stored in directory: /Users/rissacao/Library/Caches/pip/wheels/12/e5/b5/90c69cc81edf5b95cbec373c83c3530266fdae8266037380df
Successfully built ibm-watson ibm-cloud-sdk-core
Note: you may need to restart the kernel to use updated packages.


In [194]:
#obtain API key from (https://cloud.ibm.com/services/natural-language-understanding/crn%3Av1%3Abluemix%3Apublic%3Anatural-language-understanding%3Aus-south%3Aa%2Fd233267034ac49f0b942af45cf16b07f%3Aa60ba095-298a-419b-a77c-7e21faba55d1%3A%3A?paneId=credentials&new=true) 
#write Python API calls to extract Company or Organization entities from  the article chosen in Step 1:

import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('FuXAct3C-cU0bI-XYf6agqyHh7PL6abtMa2YtN7-30hg')
service = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator)
service.set_service_url('https://gateway.watsonplatform.net/natural-language-understanding/api')

response = service.analyze(
    url='https://www.readingeagle.com/news/article/exeter-township-woman-began-career-in-computer-programming-in-1962',
    features=Features(entities=EntitiesOptions())).get_result()


print(json.dumps(response, indent=2))



{
  "usage": {
    "text_units": 1,
    "text_characters": 5729,
    "features": 1
  },
  "retrieved_url": "https://www.readingeagle.com/news/article/exeter-township-woman-began-career-in-computer-programming-in-1962",
  "language": "en",
  "entities": [
    {
      "type": "Person",
      "text": "Fister",
      "relevance": 0.978348,
      "count": 19,
      "confidence": 1
    },
    {
      "type": "Location",
      "text": "Exeter Township",
      "relevance": 0.358766,
      "count": 2,
      "confidence": 0.999045
    },
    {
      "type": "Company",
      "text": "IBM",
      "relevance": 0.226842,
      "disambiguation": {
        "subtype": [
          "SoftwareLicense",
          "Organization",
          "OperatingSystemDeveloper",
          "ProcessorManufacturer",
          "SoftwareDeveloper",
          "CompanyFounder",
          "ProgrammingLanguageDesigner",
          "ProgrammingLanguageDeveloper"
        ],
        "name": "IBM",
        "dbpedia_resource": "http:/

In [196]:
#4.Download Crunchbase Open Data Map CSV file (366MB) and store it in a directory on your computer
!pip install pyspark

     |████████████████████████████████| 215.7MB 107kB/s  eta 0:00:01   |██▍                             | 15.8MB 2.9MB/s eta 0:01:10     |█████████                       | 60.0MB 23.2MB/s eta 0:00:07     |██████████████▌                 | 98.1MB 10.0MB/s eta 0:00:12     |████████████████                | 107.8MB 8.9MB/s eta 0:00:13     |████████████████████▉           | 140.4MB 21.6MB/s eta 0:00:04     |██████████████████████▉         | 153.6MB 21.6MB/s eta 0:00:03     |███████████████████████▍        | 157.5MB 21.6MB/s eta 0:00:03     |████████████████████████▍       | 164.5MB 21.6MB/s eta 0:00:03     |█████████████████████████▎      | 170.5MB 20.7MB/s eta 0:00:03     |████████████████████████████    | 189.1MB 20.7MB/s eta 0:00:02     |████████████████████████████▌   | 192.2MB 20.7MB/s eta 0:00:02
     |████████████████████████████████| 204kB 27.8MB/s eta 0:00:01
  Stored in directory: /Users/rissacao/Library/Caches/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b47

In [197]:
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext() 
config = sc.getConf() 
sqlContext = SQLContext(sc)

print("Using Apache Spark Version", sc.version)


Using Apache Spark Version 2.4.4


In [206]:
pwd

'/Users/rissacao/Downloads/5430'

In [204]:
#load Crunchbase Open Data Map into notebook 
#by modifying the path .csv(".../...") to the file on your computer 
#where you stored the downloaded CSV file from Step 4.


df = sqlContext.read.option("header", "true").option("delimiter", ",") \
                    .option("inferSchema", "true") \
                    .csv("/Users/rissacao/Downloads/5430/cb_odm_092419.csv")
df.count()

687755

In [ ]:
#match_df = df[df['name'].contains("Disney")]
match_df = df.filter(df.homepage_domain.like('%columbia.edu%'))
#match_df = df.filter(df.short_description.like('%electric car%'))
match_df['crunchbase_uuid','name','homepage_domain','stock_symbol'].show(10, truncate=False)


In [214]:
#find matches of Company or Organization entities identified in Step 3 using rlike function and print results

#organization 1: IBM
match_df = df[df['name'].rlike("IBM")& df['homepage_domain'].rlike("ibm.com")]
match_df['crunchbase_uuid','name','homepage_domain','stock_symbol'].show(10, truncate=False)


+------------------------------------+----------------------+---------------+------------+
|crunchbase_uuid                     |name                  |homepage_domain|stock_symbol|
+------------------------------------+----------------------+---------------+------------+
|fc5fbe7a-562d-d577-b05e-46797caa134c|IBM                   |ibm.com        |swx:IBM     |
|fc5fbe7a-562d-d577-b05e-46797caa134c|IBM                   |ibm.com        |lse:IBM     |
|fc5fbe7a-562d-d577-b05e-46797caa134c|IBM                   |ibm.com        |nyse:IBM    |
|fc5fbe7a-562d-d577-b05e-46797caa134c|IBM                   |ibm.com        |fra:IBM     |
|3efd1ca9-5434-5feb-942c-5f3347a8b81a|IBM Global Services   |ibm.com        |:           |
|097740ba-25f9-d4d6-fb7e-e76ca8f7960e|IBM Security Services |ibm.com        |:           |
|b16a7278-55aa-9fbf-2c33-5af6bde83b30|IBM Research          |ibm.com        |:           |
|edbd4d6e-f235-f70d-4f4c-48e521078f9f|IBM United Kingdom Ltd|ibm.com        |:           |

In [217]:
#organization 2: Microsoft
match_df = df[df['name'].rlike("Microsoft")& df['homepage_domain'].rlike("Microsoft.com")]
match_df['crunchbase_uuid','name','homepage_domain','stock_symbol'].show(10, truncate=False)


+---------------+----+---------------+------------+
|crunchbase_uuid|name|homepage_domain|stock_symbol|
+---------------+----+---------------+------------+
+---------------+----+---------------+------------+

